In [ ]:
import openai
import os
from nsepython import nsefetch
from typing import Any
import json
from pydantic import BaseModel, Field


base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"


def fetch_price(symbol: str):
    '''
    Get the current price of a security, just pass in the ticker symbol.
    '''
    try:
        # Get quote from NSE directly
        url = f'https://www.nseindia.com/api/quote-equity?symbol={symbol}'
        data = nsefetch(url)
        
        price,percent_change = data['priceInfo']['lastPrice'],data['priceInfo']['pChange']
        return round(price, 2)
    except Exception as e:
        print(f"Error: {e}")
        return None
    


class GetFetchPriceArgs(BaseModel):
    symbol: str = Field(...,description="The ticker symbol of the security being fetched, just pass in the ticker symbol and not the full name e.g. INFY, RELIANCE ")

schema = [{
    'type':'function',
    "function":{
        "name":"fetch_price",
        "description":"Get the current price of a security, just pass in the ticker symbol.",
        "parameters": GetFetchPriceArgs.model_json_schema()
    }
}
]

class Agent:
    def __init__(self, client,system:str="",tools:list = None) -> None :
        self.client = client
        self.system = system
        self.messages : list = []
        self.tools = tools if tools is not None else []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self,message = "", *args: Any, **kwds: Any) -> Any:
        if message:
            self.messages.append({"role": "user", "content": message})
        
        final_assistant_content = self.execute()
        
        if message:
            self.messages.append({"role": "assistant", "content": final_assistant_content})

        return final_assistant_content
    
    def execute(self):
        while True:
            completion = self.client.chat.completions.create(
                model="gemini-2.5-flash-lite",
                messages = self.messages,
                tools = self.tools,
                tool_choice = "auto"
            )
            response_message = completion.choices[0].message

            if response_message.tool_calls:
                self.messages.append(response_message)

                tool_outputs = []

                for tool_call in response_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)

                    if function_name in globals() and callable(globals()[function_name]):
                        function_to_call = globals()[function_name]
                        exected_output = function_to_call(**function_args)
                        tool_output_content = str(exected_output)
                        print(f"\nExecuting tool:{function_name} with args {function_args}, Output:{tool_output_content[:500]}") #For Debugging

                        tool_outputs.append(
                            {
                                "tool_call_id":tool_call.id,
                                "role":"tool",
                                "name":function_name,
                                "content":tool_output_content
                            }
                        )
                self.messages.extend(tool_outputs)
            
            else:
                return response_message.content
    
client = openai.OpenAI(api_key=os.getenv("GEMINI_API_KEY"),base_url=base_url)
system  = "You are helpful AI assistant."
tools = schema
agent = Agent(client, system, tools)

# Example usage
prompt = str(input("\nEnter your prompt: "))
response = agent(prompt)
print("\n" + response) 


In [8]:
message = {"tool_call":{
        "id": "call_99999def",
        "type": "function",
        "function": {
            "name": "send_email",
            "arguments": "{\"to\":\"bob@email.com\",\"body\":\"Hi bob\"}"
        }}
    }
function_name = message['tool_call']['function']['name']
function_args = json.loads(message['tool_call']['function']['arguments'])

In [28]:
def send_email(to,body):
    return f"Email sent to {to} with body: {body}"
TOOLS = {
    "send_email": send_email,
}

def func(name):
    return TOOLS[name]

func(function_name)(**function_args)

'Email sent to bob@email.com with body: Hi bob'

In [29]:
print(function_args)
print(function_name)

{'to': 'bob@email.com', 'body': 'Hi bob'}
send_email
